# Ciminion2 Polynomial Model Demonstration
Usage example of the Ciminion_2 polynomial model.

In [1]:
using Oscar
include("Ciminion_2.jl")
include("Ciminion_2_polynomial_model.jl")

generate_Ciminion_2_polynomials (generic function with 1 method)

## Ciminion2 Instance

In [2]:
K = GF(10007)
rounds_C = 3
rounds_E = 3

ciminion_2 = Ciminion_2_constructor(field=K, 
                                    rounds_C=rounds_C, 
                                    rounds_E=rounds_E);

Ciminion parameters
Field: Prime field of characteristic 10007
Rounds_C: 3
Rounds_E: 3
Constants_C: [6522 764 9310; 223 1946 5499; 3988 8144 9937; 1814 9653 7588]
Constants_E: [2130 1462 3355; 8699 5274 4662; 4846 6459 7917; 9745 1768 4019]


In [3]:
plain = zero_matrix(K, 2, 1)
plain[1, 1] = rand(K)
plain[2, 1] = rand(K)
plain

[1402]
[2636]

In [4]:
key = zero_matrix(K, 2, 1)
key[1, 1] = rand(K)
key[2, 1] = rand(K)
key

[5250]
[6037]

In [5]:
nonce = rand(K)
nonce

126

In [6]:
cipher = encrypt(plain, 
                 key, 
                 nonce, 
                 ciminion_2)

[5799]
[7343]

In [7]:
decrypt(cipher, 
        key, 
        nonce, 
        ciminion_2)

[1402]
[2636]

In [8]:
plain == decrypt(cipher, key, nonce, ciminion_2)

true

## Ciminion2 Polynomial model
Generate the Ciminion_2 polynomial model, and compute a DRL Gröbner basis.

In [9]:
polys = generate_Ciminion_2_polynomials(ciminion_2=ciminion_2,
                                        plain=plain,
                                        cipher=cipher);

sep = repeat("-", 70)
println(sep)

for i in 1:(rounds_C + rounds_E)
    for j in 1:3
        println(polys[3 * (i - 1) + j])
    end
    println(sep)
end

----------------------------------------------------------------------
10006*x_C_1__1 + 4568*y_1 + y_2 + 6522
10006*x_C_2__1 + 2370*y_1 + 1814*y_2 + 4791
10006*x_C_3__1 + 4569*y_1 + y_2 + 3988
----------------------------------------------------------------------
x_C_1__1*x_C_2__1 + x_C_3__1 + 10006*x_C_1__2 + y_1 + y_2 + 764
9653*x_C_1__1*x_C_2__1 + x_C_1__1 + 9653*x_C_2__1 + 9653*x_C_3__1 + 10006*x_C_2__2 + 9653*y_1 + 9653*y_2 + 1946
x_C_1__1*x_C_2__1 + x_C_2__1 + x_C_3__1 + 10006*x_C_3__2 + y_1 + y_2 + 8144
----------------------------------------------------------------------
x_C_1__2*x_C_2__2 + x_C_3__2 + 10006*x_E_1__1 + y_1 + y_2 + 9310
7588*x_C_1__2*x_C_2__2 + x_C_1__2 + 7588*x_C_2__2 + 7588*x_C_3__2 + 10006*x_E_2__1 + 7588*y_1 + 7588*y_2 + 5499
x_C_1__2*x_C_2__2 + x_C_2__2 + x_C_3__2 + 10006*x_E_3__1 + y_1 + y_2 + 9937
----------------------------------------------------------------------
x_E_1__1*x_E_2__1 + x_E_3__1 + 10006*x_E_1__2 + y_1 + y_2 + 2130
9745*x_E_1__1*x_E_2__1 +

Generate the efficient DRL term order which produces a quadratic DRL Gröbner basis.

In [10]:
P = parent(polys[1])
variables = gens(P)

variables_Q = String[]
for var in variables
    push!(variables_Q, string(var))
end

variables_Q = [
variables_Q[3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 1:3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 2] 
variables_Q[1:3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E)];
variables_Q[3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 3:3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 3]
]

Q, variables_Q = polynomial_ring(base_ring(P), 
                                 variables_Q, 
                                 internal_ordering=:degrevlex);

Generate a homomorphism between $P$ and $Q$.

In [11]:
variables_tmp = [
variables_Q[2 + 1:2 + 3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E)];
variables_Q[1:2];
variables_Q[3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 3:3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 3]
]

h = hom(P, Q, variables_tmp)

Ring homomorphism
  from multivariate polynomial ring in 18 variables over K
  to multivariate polynomial ring in 18 variables over K
defined by
  x_C_1__1 -> x_C_1__1
  x_C_2__1 -> x_C_2__1
  x_C_3__1 -> x_C_3__1
  x_C_1__2 -> x_C_1__2
  x_C_2__2 -> x_C_2__2
  x_C_3__2 -> x_C_3__2
  x_E_1__1 -> x_E_1__1
  x_E_2__1 -> x_E_2__1
  x_E_3__1 -> x_E_3__1
  x_E_1__2 -> x_E_1__2
  x_E_2__2 -> x_E_2__2
  x_E_3__2 -> x_E_3__2
  x_E_1__3 -> x_E_1__3
  x_E_2__3 -> x_E_2__3
  x_E_3__3 -> x_E_3__3
  y_1 -> y_1
  y_2 -> y_2
  x -> x

Now we compute the DRL Gröbner basis with respect to the efficient term order.

In [12]:
gb_Q = groebner_basis_f4(h(ideal(polys)), nr_thrds=16, info_level=2);


Legend for f4 information
--------------------------------------------------------
deg       current degree of pairs selected in this round
sel       number of pairs selected in this round
pairs     total number of pairs in pair list
mat       matrix dimensions (# rows x # columns)
density   density of the matrix
new data  # new elements for basis in this round
          # zero reductions during linear algebra
time(rd)  time of the current f4 round in seconds given
          for real and cpu time
--------------------------------------------------------

deg     sel   pairs        mat          density            new data         time(rd) in sec (real|cpu)
------------------------------------------------------------------------------------------------------
  1       2      12       3 x 6          66.67%        2 new       0 zero         0.01 | 0.23         
  2      11      11      19 x 25         24.63%       11 new       0 zero         0.04 | 0.45         
  2       5       5      36


--------------- INPUT DATA ---------------
#variables                      18
#equations                      18
#invalid equations               0
field characteristic         10007
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                        16
info level                       2
generate pbm files               0
------------------------------------------

---------------- TIMINGS ----------------
overall(elapsed)        0.12 sec
overall(cpu)            1.71 sec
select                  0.00 sec   0.0%
symbolic prep.          0.00 sec   0.0%
update                  0.00 sec   0.0%
convert                 0.00 sec   0.0%
linear algebra          0.00 sec   0.0%
reduce gb               0.00 sec   0.0%
--------------------------------

In [13]:
for poly in gb_Q
    println(poly)
    println(sep)
end

x_E_2__3 + 10006*x + 8959
----------------------------------------------------------------------
x_E_1__3 + 4019*x + 3792
----------------------------------------------------------------------
x_E_2__2 + 10006*x_E_3__3 + 5988*x + 1205
----------------------------------------------------------------------
x_E_1__2 + 1768*x_E_3__3 + 10006*x + 2701
----------------------------------------------------------------------
x_E_2__1 + 10006*x_E_3__2 + 8239*x_E_3__3 + x + 15
----------------------------------------------------------------------
x_E_1__1 + 9745*x_E_3__2 + 10006*x_E_3__3 + 5988*x + 8667
----------------------------------------------------------------------
x_C_2__2 + 10006*x_E_3__1 + 262*x_E_3__2 + x_E_3__3 + 4019*x + 1967
----------------------------------------------------------------------
x_C_1__2 + 7588*x_E_3__1 + 10006*x_E_3__2 + 8239*x_E_3__3 + x + 6303
----------------------------------------------------------------------
x_C_3__1 + 7839*x_C_3__2 + 724*x_E_3__1 + 2168*x_E_